# How to use the converter utility

This tool converts the input internally calibrated mean spectra from the continuous representation to a sampled form.

## Basic usage

In [ ]:
# Import the tool
from gaiaxpy import convert
# Path to file with XP CONTINUOUS RAW data (csv, ecsv, fits, or xml)
f = '/path/to/XP_CONTINUOUS_RAW.fits'

#### The converter returns two different outputs: 

 1. The values for all sampled spectra as a pandas DataFrame.
 2. The sampling used to convert the input spectra in pseudo-wavelength. This sampling should be an iterable (e.g. a NumPy array), the minimum accepted value is -10 and the maximum is 70. The default sampling is numpy.linspace(0, 60, 600).

In [ ]:
converted_spectra, sampling = convert(f)
converted_spectra

### Input types

The input does not have to be a path to a file. There are other options: a pandas DataFrame, an ADQL query or a list of sourceIds.

#### Passing a DataFrame

In [ ]:
import pandas as pd

f = '/path/to/XP_CONTINUOUS_RAW.csv'
df = pd.read_csv(f) # The values in the DataFrame can be edited if the user wishes to do so.

converted_spectra, sampling = convert(df)
converted_spectra

#### Running a query

**Important note**: Currently the data in the Gaia Archive is not public. The following code will <span style="color:red">not</span> work until DR3.

In [ ]:
query_input = "select TOP 2 source_id from gaiadr3.gaia_source where has_xp_continuous = 'True'"
converted_spectra, sampling = convert(query_input)
converted_spectra

#### Passing a list

A list of sourceIds can be passed to the converter as the first argument. The converter will then query the Archive for these objects.

In [ ]:
sources_list = ['48', 44] # The sourceIds can be string or long.
converted_spectra, sampling = convert(sources_list)
converted_spectra

## Advanced usage

Additional arguments can be passed to the converter.

These are:
1. sampling
2. truncation
3. output_path
3. output_file
4. output_format
5. save_file

### Sampling

A custom sampling can be passed to the function. This sampling should be an iterable (list, tuple, generator, or preferably a NumPy array). If no sampling is given, the default sampling is returned.

The default sampling of the converter corresponds to numpy.linspace(0., 60., 600.) in pseudo-wavelength.

The minimum value allowed in the sampling is -10, and the maximum is 70. The program will raise an error is the sampling does not comply.

In [ ]:
import numpy as np

converted_spectra_default, default_sampling = convert(df.iloc[[0]])
converted_spectra, custom_sampling = convert(f, sampling=np.arange(-10, 70, 0.5))
converted_spectra

The results can be plotted using the plotter utility.

In [ ]:
from gaiaxpy import plot_spectra
# multi allows the spectra to appear in the same canvas
plot_spectra(converted_spectra, sampling=custom_sampling, multi=True)

The figure can be saved too.

In [ ]:
plot_spectra(converted_spectra, sampling=custom_sampling, multi=True, output_path='/path/to', show_plot=False)

### Truncation

The source mean BP/RP spectrum is described as a combination of basis functions. Particularly for faint sources or sources with a low number of observations, it is useful to represent the spectrum using a smaller set of basis functions to avoid higher-order bases fitting the noise in the observed data.

The truncation parameter is a boolean which toggles the truncation of the set of bases.

In [ ]:
first_source = df.drop([0]) # We'll use only the first source (sourceId 4) in the demonstration
non_truncated_spectra, sampling = convert(first_source) # truncation is False by default
truncated_spectra, _ = convert(first_source, truncation=True)

We can use GaiaXPy's plot spectra utility to see the differences in the results.

In [ ]:
from gaiaxpy import plot_spectra
# Construct a DataFrame so we can plot both sources using just one function call
data = pd.concat([non_truncated_spectra, truncated_spectra]).reset_index(drop=True)
# As both spectra we want to plot have the same sourceId, we'll rename them to get descriptive labels
data['source_id'][0:2] = 'Non truncated'
data['source_id'][2:4] = 'Truncated'
plot_spectra(data, sampling=sampling, multi=True)

The differences between the truncated and not truncated versions are too small to be seen in the previous plot. But we can compare the values.

In [ ]:
# We'll get the flux values of the BP band of the first spectrum in the data
non_truncated_bp_flux = data[(data['source_id'] == 'Non truncated') & (data['xp'] == 'BP')]['flux'].iloc[0]
truncated_bp_flux = data[(data['source_id'] == 'Truncated') & (data['xp'] == 'BP')]['flux'].iloc[0]
# We can then compute the absolute differences
abs_differences = np.abs(non_truncated_bp_flux - truncated_bp_flux)
# And find the values and indices of those differences
min_diff = np.min(abs_differences)
max_diff = np.max(abs_differences)
print(f'The length of the sampling is {len(non_truncated_bp_flux)}.')
print(f'Minimum difference: {min_diff}. Index of minimum difference: {np.where(abs_differences == min_diff)[0][0]}.')
print(f'Maximum difference: {max_diff}. Index of maximum difference: {np.where(abs_differences == max_diff)[0][0]}.')

### Output_path, output_file, output_format, save_file

Three parameters: **output_path**, **output_file**, and **output_format** define the entire path of the resulting file.

The default output path is the current path. If the given output path does not exist, it will be created.

The default output file name is 'output_spectra'. 

The default output format is the format of the input file (i.e. if the input file is a 'fits', then the output file will be a FITS file by default.), or CSV in any other case (DataFrame, ADQL query or list).

**NOTE: If a file with the same path and name already exists, it will be AUTOMATICALLY OVERWRITTEN.**

In [ ]:
converted_spectra, _ = convert(f, output_path='/path/to', output_file='my_file', output_format='ecsv')

The additional parameter **save_file** is a boolean that tells the program whether to save the results or not.
If 'output_file' is given but 'save_file' is set to False, a warning will be raised.

In [ ]:
converted_spectra, _ = convert(f, output_file='my_file', output_format='.csv', save_file=False)